In [ ]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from keras_tuner import HyperModel, RandomSearch
from tensorflow.keras.optimizers import Adam, AdamW
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.layers import Input

In [ ]:
# Load dataset
data = pd.read_csv("task_dataset.csv")
data.head()

,Timestamp,Nama Tugas,Deadline Tugas,Tingkat Kesulitan Tugas,Estimasi Durasi Pengerjaan Tugas,Kategori Tugas,Prioritas Tugas,Email Address
0,11/23/2024 11:42:01,Laporan Analisis Performa Penjualan dan Strate...,11/24/2024,2,Lebih dari 6 Jam,"Akademik (Tugas Kuliah, Laporan Praktikum, dll.)",5 (Sangat Prioritas),NaN
1,11/23/2024 11:53:00,Artikel Ilmiah Penelitian,11/25/2024,2,4-6 Jam,"Akademik (Tugas Kuliah, Laporan Praktikum, dll.)",4 (Penting),NaN
2,11/23/2024 12:06:11,capstone,12/12/2024,2,Lebih dari 6 Jam,"Akademik (Tugas Kuliah, Laporan Praktikum, dll.)",5 (Sangat Prioritas),NaN
3,11/23/2024 12:07:12,Laporan Study Field,11/28/2024,2,4-6 Jam,"Akademik (Tugas Kuliah, Laporan Praktikum, dll.)",4 (Penting),NaN
4,11/23/2024 12:25:34,Membuat LPJ SRD,11/24/2024,3,4-6 Jam,"Organisasi (Kepengurusan, Kegiatan Organisasi,...",1 (Tidak Prioritas),NaN


In [ ]:
# Clean the column names by removing extra spaces
data.columns = data.columns.str.strip()

# Rename columns
data.rename(columns={
    'Timestamp': 'timestamp',
    'Nama Tugas': 'task_name',
    'Deadline Tugas': 'deadline',
    'Tingkat Kesulitan Tugas': 'difficulty_level',
    'Estimasi Durasi Pengerjaan Tugas': 'duration',
    'Kategori Tugas': 'task_category',
    'Prioritas Tugas': 'priority_level',
    'Email Address': 'email'
}, inplace=True)

# Convert 'timestamp' and 'deadline' columns to datetime
data['timestamp'] = pd.to_datetime(data['timestamp'])
data['deadline'] = pd.to_datetime(data['deadline'])

# Create a new column for the difference in days between 'deadline' and 'timestamp'
data['days_until_deadline'] = (data['deadline'] - data['timestamp']).dt.days

# Map 'duration' categories to numeric values
duration_mapping = {
    "Kurang dari 1 Jam": 1,
    "1-3 Jam": 3,
    "4-6 Jam": 5,
    "Lebih dari 6 Jam": 7
}
data['duration'] = data['duration'].map(duration_mapping)

# Clean column to extract numeric values
data['priority_level'] = data['priority_level'].str.extract('(\d)').astype(int)

# Drop column
data.drop(columns=['email'], inplace=True)

# Convert categorical columns to strings
data['task_category'] = data['task_category'].astype(str)

# Feature Engineering: Create more features
data['hour_of_day'] = data['timestamp'].dt.hour
data['day_of_week'] = data['timestamp'].dt.dayofweek

In [ ]:
X = data[['days_until_deadline', 'difficulty_level', 'duration', 'hour_of_day', 'day_of_week']].values
y = data['priority_level'].values

In [ ]:
# Data normalization
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# Split data into train, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
# Scale target variable
y_scaler = MinMaxScaler()
y_train_scaled = y_scaler.fit_transform(y_train.reshape(-1, 1))
y_val_scaled = y_scaler.transform(y_val.reshape(-1, 1))
y_test_scaled = y_scaler.transform(y_test.reshape(-1, 1))

In [ ]:
# Define a custom Learning Rate Scheduler
def lr_scheduler(epoch, lr):
    if epoch % 10 == 0 and epoch != 0:
        lr = lr * 0.5  # Reduce learning rate by half every 10 epochs
    return lr

In [ ]:
# Define Hypermodel for Keras Tuner
class HyperModelExample(HyperModel):
    def build(self, hp):
        model = Sequential()
        model.add(Input(shape=(X_train.shape[1],)))
        model.add(Dense(
            units=hp.Int('units_1', min_value=64, max_value=256, step=64),
            activation='relu'
        ))
        model.add(Dropout(hp.Float('dropout_1', min_value=0.2, max_value=0.5, step=0.1)))
        model.add(Dense(
            units=hp.Int('units_2', min_value=64, max_value=128, step=64),
            activation='relu'
        ))
        model.add(Dropout(hp.Float('dropout_2', min_value=0.2, max_value=0.5, step=0.1)))
        model.add(Dense(1))

        model.compile(
            optimizer=AdamW(learning_rate=hp.Float('learning_rate', min_value=1e-5, max_value=1e-3, sampling='log')),
            loss='mse',
            metrics=['mae']
        )

        return model

In [ ]:
# Hyperparameter search using RandomSearch
tuner = RandomSearch(
    HyperModelExample(),
    objective='val_loss',
    max_trials=20,  # Increased the number of trials for more thorough search
    executions_per_trial=3,  # Multiple executions for more robust results
    directory='project_dir',
    project_name='task_priority_optimization'
)

In [ ]:
# Running the hyperparameter search
tuner.search(X_train, y_train_scaled, epochs=20, validation_data=(X_val, y_val_scaled))

Trial 20 Complete [00h 00m 14s]
val_loss: 0.04203140611449877

Best val_loss So Far: 0.04203140611449877
Total elapsed time: 00h 04m 33s


In [ ]:
best_trial = tuner.oracle.get_best_trials(1)[0]

In [ ]:
best_model = tuner.hypermodel.build(best_trial.hyperparameters)

In [ ]:
best_model.fit(X_train, y_train_scaled, epochs=20, validation_data=(X_val, y_val_scaled))

Epoch 1/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - loss: 0.5915 - mae: 0.7126 - val_loss: 0.2731 - val_mae: 0.4798
Epoch 2/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.2390 - mae: 0.4215 - val_loss: 0.0813 - val_mae: 0.2343
Epoch 3/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0906 - mae: 0.2548 - val_loss: 0.0621 - val_mae: 0.1968
Epoch 4/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.1165 - mae: 0.2675 - val_loss: 0.0796 - val_mae: 0.2184
Epoch 5/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.1376 - mae: 0.2762 - val_loss: 0.0638 - val_mae: 0.1924
Epoch 6/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0999 - mae: 0.2410 - val_loss: 0.0512 - val_mae: 0.1900
Epoch 7/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0693 - mae: 0.2155 - val_loss: 0.0538 - val_mae: 0.1952
Epoch 8/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0760 - mae: 0.2209 - val_loss: 0.0604 - val_mae: 0.2045
Epoch 9/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0769 - mae: 0.2345 - 

In [ ]:
test_loss, test_mae = best_model.evaluate(X_test, y_test_scaled, verbose=1)
print(f"Test Loss: {test_loss}")
print(f"Test MAE: {test_mae}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0714 - mae: 0.2206 
Test Loss: 0.06953588873147964
Test MAE: 0.2176869660615921


In [ ]:
# Save the trained model
best_model.save('model.h5')

In [ ]:
import pickle

# Save the model with pickle
with open('model.pkl', 'wb') as f:
    pickle.dump(best_model, f)

In [ ]:
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.models import load_model

# Memuat model yang disimpan
model = load_model('model.h5', custom_objects={'mse': MeanSquaredError()})

In [ ]:
# Data Tugas
tasks = [
    {
        'timestamp': '11/23/2024 13:15:35',
        'task_name': 'Project Machine Learning',
        'deadline': '11/30/2024',
        'difficulty_level': 3,
        'duration': 'Lebih dari 6 Jam'
    },
    {
        'timestamp': '11/23/2024 13:22:36',
        'task_name': 'Laporan Praktikum Perancangan Fasilitas Pembuatan Trolley',
        'deadline': '12/7/2024',
        'difficulty_level': 3,
        'duration': 'Lebih dari 6 Jam'
    },
    {
        'timestamp': '11/23/2024 13:24:20',
        'task_name': 'Makalah Perilaku Organisasi',
        'deadline': '11/25/2024',
        'difficulty_level': 1,
        'duration': '4-6 Jam'
    },
    {
        'timestamp': '11/23/2024 14:26:14',
        'task_name': 'Metpen',
        'deadline': '11/30/2024',
        'difficulty_level': 2,
        'duration': '4-6 Jam'
    },
    {
        'timestamp': '11/23/2024 15:38:42',
        'task_name': 'Simulasi Rangkaian Sistem Digital dengan Aplikasi Proteus',
        'deadline': '11/26/2024',
        'difficulty_level': 2,
        'duration': '1-3 Jam'
    },
    {
        'timestamp': '11/23/2024 20:06:32',
        'task_name': 'Laporan Audit Sistem Informasi',
        'deadline': '12/18/2024',
        'difficulty_level': 3,
        'duration': '1-3 Jam'
    },
    {
        'timestamp': '11/23/2024 20:08:28',
        'task_name': 'Review Jurnal Desain Manajemen Jaringan',
        'deadline': '11/25/2024',
        'difficulty_level': 1,
        'duration': '1-3 Jam'
    }
]

# Mapping durasi ke nilai numerik
duration_mapping = {
    "Kurang dari 1 Jam": 1,
    "1-3 Jam": 3,
    "4-6 Jam": 5,
    "Lebih dari 6 Jam": 7
}

# Proses data
processed_tasks = []
for task in tasks:
    task['timestamp'] = pd.to_datetime(task['timestamp'])
    task['deadline'] = pd.to_datetime(task['deadline'])
    task['days_until_deadline'] = (task['deadline'] - task['timestamp']).days  # Perbaikan di sini
    task['hour_of_day'] = task['timestamp'].hour
    task['day_of_week'] = task['timestamp'].dayofweek
    task['duration'] = duration_mapping[task['duration']]
    processed_tasks.append([
        task['days_until_deadline'],
        task['difficulty_level'],
        task['duration'],
        task['hour_of_day'],
        task['day_of_week']
    ])

# Normalisasi input
tasks_input = scaler.transform(processed_tasks)

# Prediksi prioritas untuk setiap tugas
predicted_priorities_scaled = best_model.predict(tasks_input)
predicted_priorities = y_scaler.inverse_transform(predicted_priorities_scaled)

# Gabungkan hasil prediksi dengan data tugas
for i, task in enumerate(tasks):
    task['predicted_priority'] = predicted_priorities[i][0]

# Urutkan tugas berdasarkan prioritas prediksi
tasks_sorted = sorted(tasks, key=lambda x: x['predicted_priority'], reverse=True)

# Cetak hasil urutan prioritas
print("Urutan Tugas Berdasarkan Prioritas (Tinggi ke Rendah):")
for i, task in enumerate(tasks_sorted):
    print(f"{i + 1}. {task['task_name']} | Deadline: {task['deadline']} | "
          f"Difficulty: {task['difficulty_level']} | Duration: {task['duration']} | "
          f"Predicted Priority: {task['predicted_priority']:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Urutan Tugas Berdasarkan Prioritas (Tinggi ke Rendah):
1. Laporan Praktikum Perancangan Fasilitas Pembuatan Trolley | Deadline: 2024-12-07 00:00:00 | Difficulty: 3 | Duration: 7 | Predicted Priority: 4.23
2. Project Machine Learning | Deadline: 2024-11-30 00:00:00 | Difficulty: 3 | Duration: 7 | Predicted Priority: 4.22
3. Laporan Audit Sistem Informasi | Deadline: 2024-12-18 00:00:00 | Difficulty: 3 | Duration: 3 | Predicted Priority: 4.13
4. Metpen | Deadline: 2024-11-30 00:00:00 | Difficulty: 2 | Duration: 5 | Predicted Priority: 3.56
5. Simulasi Rangkaian Sistem Digital dengan Aplikasi Proteus | Deadline: 2024-11-26 00:00:00 | Difficulty: 2 | Duration: 3 | Predicted Priority: 3.44
6. Review Jurnal Desain Manajemen Jaringan | Deadline: 2024-11-25 00:00:00 | Difficulty: 1 | Duration: 3 | Predicted Priority: 3.36
7. Makalah Perilaku Organisasi | Deadline: 2024-11-25 00:00:00 | Difficulty: 1 | Duration: 5 | Predicted Priority: 3.34
